In [1]:
import pandas as pd
from RoutePlanner.CellGrid import CellGrid

#read in Ice  & Currents Points
icePoints     = pd.read_json('../resources/icepoints2019-02-08.json')
icePoints     = pd.DataFrame.from_records(icePoints.icepoints)
currentPoints = pd.read_json('../resources/currentPoints.json')
currentPoints = pd.DataFrame.from_records(currentPoints.currentPoints)

longMin    = 260
longMax    = 360
latMin     = -75
latMax     = -40
cellWidth  = 5
cellHeight = 2.5

cellGrid = CellGrid(longMin, longMax, latMin, latMax, cellWidth, cellHeight)
cellGrid.addIcePoints(icePoints)
cellGrid.addCurrentPoints(currentPoints)

from RoutePlanner.Optimisation import TravelTime
OptInfo = {}
OptInfo['WayPoints']            = pd.read_csv('../resources/WayPoints_TestExample2.csv') 
OptInfo['MaxIceExtent']         = 0.8
OptInfo['VehicleInfo']          = {}
OptInfo['VehicleInfo']['Speed'] = 26.3 #Units km/hr

TT = TravelTime(cellGrid,OptInfo)
TT.Dijkstra(verbrose=True)



=== Processing Waypoint = Falklands ===
=== Processing Waypoint = SouthGeorgia ===


In [2]:
from RoutePlanner.Function import NewtonianCurve

In [4]:
import numpy as np
from tqdm import tqdm

Path = TT.Paths[1]


startPoint = Path['Path']['FullPath'][0,:][None,:]
endPoint   = Path['Path']['FullPath'][-1,:][None,:]

OrgcrossingPoints = np.concatenate([startPoint,
                                 Path['Path']['CrossingPoints'],
                                 endPoint])

Points = OrgcrossingPoints.copy()
iter=0
for iter in tqdm(range(50)):
    for id in range(Points.shape[0]-2):
        Sp  = tuple(Points[id,:])
        Cp  = tuple(Points[id+1,:])
        Np  = tuple(Points[id+2,:])
        nc = NewtonianCurve(TT.Mesh,Sp,Cp,Np,TT.OptInfo['VehicleInfo']['Speed'],debugging=1,maxiter=4)
        TravelTime, CrossingPoint = nc.value()

        #Points[id+1,:] = CrossingPoint

        if id == 0:
            newPoints = CrossingPoint
        else:
            newPoints = np.concatenate([newPoints,CrossingPoint])

    Points = np.concatenate([startPoint,newPoints,endPoint])


  0%|          | 0/50 [00:00<?, ?it/s]


TypeError: cannot unpack non-iterable float object

In [ ]:
ax = cellGrid.plot(return_ax=True)

crossingPoints = np.concatenate([Path['Path']['FullPath'][0,:][None,:],
                                 Path['Path']['CrossingPoints'],
                                 Path['Path']['FullPath'][-1,:][None,:]])


ax.scatter(Path['Path']['FullPath'][:,0],Path['Path']['FullPath'][:,1],15,'b')
ax.plot(Path['Path']['FullPath'][:,0],Path['Path']['FullPath'][:,1],'b')

ax.scatter(Points[:,0],Points[:,1],15,'r')
ax.plot(Points[:,0],Points[:,1],'r')


ax.scatter(305.00,-33.47)

# Loading Ground Truth & Comparing
GT = pd.read_csv('../resources/TestExample_GroundTruth2_Smooth.csv')
GT['Long'] = GT['Long']+360
ax.plot(GT['Long'],GT['Lat'],'k')
ax.scatter(GT['Long'],GT['Lat'],15,'k')

GT = pd.read_csv('../resources/TestExample_GroundTruth2.csv')
GT['Long'] = GT['Long']+360
ax.plot(GT['Long'],GT['Lat'],'m')
ax.scatter(GT['Long'],GT['Lat'],15,'m')



ax.set_xlim([295,325])
ax.set_ylim([-60,-50])